
# Prerequisite

In [ ]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset
from azure.identity import InteractiveBrowserCredential

In [ ]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(credential=InteractiveBrowserCredential(), subscription_id=subscription_id, resource_group_name=resource_group, workspace_name=workspace)

# Basic pipeline job

## Build pipeline

In [ ]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from azure.ml.entities import Component
from pathlib import Path

# 1. Load component funcs
# please make sure version in the yaml is a new one
parent_dir = '.'
a_component = Component.load(path=parent_dir + "/componentA.yml")
a_func = dsl.load_component(component=a_component)
b_component = Component.load(path=parent_dir + "/componentB.yml")
b_func = dsl.load_component(component=b_component)
c_component = Component.load(path=parent_dir + "/componentC.yml")
c_func = dsl.load_component(component=c_component)


# 2. Construct pipeline
# Define pipeline (a function decorated with dsl.pipeline), each component function called here will create a node in pipeline graph
# Parameter of this pipeline function will be mapped as a pipeline parameter

@dsl.pipeline(
    name="hello_world_pipeline",
    compute="cpu-cluster",
    description="Basic Pipeline Job with 3 Hello World components",
)
def sample_pipeline():
    componentA_job = a_func()
    componentB_job = b_func()
    componentC_job = c_func()

# Submit pipeline job

In [ ]:
# create pipeline instance
pipeline = sample_pipeline()
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="basic_pipeline", continue_run_on_step_failure=True)